In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import random
from tslearn.metrics import dtw

In [2]:
#Unsere Trainingsdaten einlesen
#Dieser Schritt wird einmalig passieren und dabei gehts um die Daten auf von denen wir dann unsere prediction basieren
#Da wir die Daten haben ist eigentlich egal ob die pickle oder csv sind deswegen nehm ich jetzt mal pickle
df = pd.read_pickle("Train.pkl")
df.timestamp = pd.to_datetime(df.timestamp)
df_new = df.loc[df["CuStepNo ValueY"] != 2] #hat er uns ja gesagt diesen Schritt zu ignorieren
df_new["Next_Step"] = df_new["CuStepNo ValueY"].shift()
df_steps = df_new.loc[df_new["CuStepNo ValueY"] != df_new["Next_Step"]]
#df_steps ist nen Hilfsdataframe um später die nötigen nummer von Batches auf die Originaldaten zu bekommen
times = df_steps[df_steps["CuStepNo ValueY"] == 1].timestamp.to_list()
df_train = pd.DataFrame(columns=df_new.columns)
#df_test2 wird unser hilfsframe wo dann die batch nummer steht
for i in range(times.__len__()-1):
    df_test1 = df_new.loc[(df_new.timestamp >=times[i])&(df_new.timestamp <times[i+1])]
    df_test1["Batch"] = i
    df_train = pd.concat([df_train,df_test1])
    

C:\Users\paulh\AppData\Local\Temp\ipykernel_20688\471522017.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test1["Batch"] = i
C:\Users\paulh\AppData\Local\Temp\ipykernel_20688\471522017.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test1["Batch"] = i
C:\Users\paulh\AppData\Local\Temp\ipykernel_20688\471522017.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [20]:
#dies ist dann unsere basis trainingsdatenset
df_train.head()

,timestamp,DeviationID ValueY,CuStepNo ValueY,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,FIC14002_PV_Out ValueY,...,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY,Next_Step,Batch
0,2018-10-10 14:53:19,1,1,149.985779,100,0.070602,0.087963,100.0,100.0,0.0,...,0.611938,1.979745,15,10.0,0.127008,0.2,0,0.00171,NaN,0.0
1,2018-10-10 14:53:20,1,1,149.985779,100,0.070602,0.086806,100.0,100.0,0.0,...,1.151489,1.979745,15,10.0,0.127021,0.2,0,0.001715,1.0,0.0
2,2018-10-10 14:53:21,1,1,150.02774,100,0.071759,0.103588,100.0,100.0,0.0,...,11.073669,1.986111,15,10.0,0.127021,0.2,0,0.001712,1.0,0.0
3,2018-10-10 14:53:22,1,1,150.043427,100,0.072338,0.173611,100.0,100.0,0.0,...,11.819598,1.982133,15,10.0,0.126945,0.2,0,0.001698,1.0,0.0
4,2018-10-10 14:53:23,1,1,150.015564,100,0.065972,0.326968,100.0,100.0,0.0,...,22.06959,1.982133,15,10.0,0.126945,0.2,0,0.0017,1.0,0.0


Damit mans testen kann hab ich paar batches aus der csv rausgeschrieben, dass man an diesen testen kann

In [3]:
#Als nächstes brauchen wir irgendeine Funktion die kontinuierlich die Daten die von der Maschine kommen aufnimmt und diese mit alten daten vereint also basically pd.concat
df_test1_new = pd.read_csv("Batch_211.csv") #name is egal aber damit mir die zeile darunter nicht dauerhaft anzeigt dass die variable nicht implementiert ist
# Dabei sei gesagt der code ist so angepasst dass es sich bei df_test1_new um immer nur die Zeilen aus einem Step handeln

In [31]:
df_test1_new

,Unnamed: 0,timestamp,DeviationID ValueY,CuStepNo ValueY,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,...,LIC22002_PV_Out ValueY,LIC22002_SP ValueY,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY
0,222636,2018-10-29 09:24:41,9,1,99.149460,100,0.052083,0.076389,100.0,100.0,...,1.988498,2,3.814819,1.994068,15,10.0,0.108480,0.2,0,0.001782
1,222637,2018-10-29 09:24:42,9,1,99.179779,100,0.053819,0.123843,100.0,100.0,...,1.990090,2,7.424255,2.017940,15,10.0,0.108516,0.2,0,0.001764
2,222638,2018-10-29 09:24:43,9,1,99.210144,100,0.052083,0.240741,100.0,100.0,...,1.994068,2,10.684339,2.068866,15,10.0,0.108548,0.2,0,0.001757
3,222639,2018-10-29 09:24:44,9,1,99.239563,100,0.049190,0.416667,100.0,100.0,...,1.994864,2,16.128729,2.056930,15,10.0,0.108577,0.2,0,0.001757
4,222640,2018-10-29 09:24:45,9,1,99.265030,100,0.049769,0.598958,100.0,100.0,...,1.994864,2,23.149170,2.035446,15,10.0,0.108584,0.2,0,0.001766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,223734,2018-10-29 09:42:59,9,3,98.538330,100,0.048611,0.065972,100.0,100.0,...,2.009187,2,0.000000,2.022714,15,10.0,0.116866,0.2,0,0.001727
1099,223735,2018-10-29 09:43:00,9,3,98.688446,100,0.054398,0.070023,100.0,100.0,...,1.995660,2,0.000000,2.018735,15,10.0,0.116962,0.2,0,0.001736
1100,223736,2018-10-29 09:43:01,9,3,98.688446,100,0.054398,0.066551,100.0,100.0,...,1.992477,2,0.000000,2.009983,15,10.0,0.117002,0.2,0,0.001726
1101,223737,2018-10-29 09:43:02,9,3,98.910507,100,0.054977,0.068287,100.0,100.0,...,1.986907,2,0.000000,2.003617,15,10.0,0.117078,0.2,0,0.001738


In [37]:
#Danach kommt unsere preditction
#Dabei geben wir immer die letzte Zahl von y_predict während eines Steps als Ausgabe aus und wenn diese nicht 1 ist soll halt etwas passieren
y_predicted = []

for l in df_test1_new["CuStepNo ValueY"].unique(): #wäre nach dem Fall von oben immer nur eine Zahl aber für den Zweck des Testens lass ichs mal so
    score = []
    for i in range(1,11):
        meanscore = []
        for j in df_train[(df_train["CuStepNo ValueY"] == 8)&(df_train["DeviationID ValueY"] == i)].Batch.unique():
            meanscore.append(dtw(df_test1_new[(df_test1_new["CuStepNo ValueY"]==l)].drop(["DeviationID ValueY","timestamp"],axis=1),df_train[(df_train["DeviationID ValueY"]==i)&(df_train["CuStepNo ValueY"]==l)&(df_train["Batch"]==j)].drop(["DeviationID ValueY","Batch","Next_Step","timestamp"],axis=1)))
        score.append(np.min(meanscore))
y_predicted.append(np.argmin(score)+1)
#y_predicted ist dann die Liste mit den Fehlerids die prognestiert werden


In [4]:
df_train.loc[df_train["Batch"] == 0].head()

,Unnamed: 0,timestamp,DeviationID ValueY,CuStepNo ValueY,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,...,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY,Next_Step,Batch
0,0,2018-10-10 14:53:19,1,1,149.985779,100,0.070602,0.087963,100.0,100.0,...,0.611938,1.979745,15,10.0,0.127008,0.2,0,0.00171,NaN,0
1,1,2018-10-10 14:53:20,1,1,149.985779,100,0.070602,0.086806,100.0,100.0,...,1.151489,1.979745,15,10.0,0.127021,0.2,0,0.001715,1.0,0
2,2,2018-10-10 14:53:21,1,1,150.02774,100,0.071759,0.103588,100.0,100.0,...,11.073669,1.986111,15,10.0,0.127021,0.2,0,0.001712,1.0,0
3,3,2018-10-10 14:53:22,1,1,150.043427,100,0.072338,0.173611,100.0,100.0,...,11.819598,1.982133,15,10.0,0.126945,0.2,0,0.001698,1.0,0
4,4,2018-10-10 14:53:23,1,1,150.015564,100,0.065972,0.326968,100.0,100.0,...,22.06959,1.982133,15,10.0,0.126945,0.2,0,0.0017,1.0,0


In [16]:
from tslearn.metrics import cdist_dtw
from tslearn.metrics import dtw_subsequence_path
from tslearn.metrics import dtw_path
from dtaidistance import dtw
from dtaidistance import dtw_ndim

In [5]:
df_2 = df_train[(df_train["CuStepNo ValueY"]==3)&(df_train["Batch"]==1)].drop(["DeviationID ValueY","Batch","Next_Step","timestamp"],axis=1).values
df_3 = df_train[(df_train["CuStepNo ValueY"]==3)&(df_train["Batch"]==0)].drop(["DeviationID ValueY","Batch","Next_Step","timestamp"],axis=1).values
df_4 = df_train[(df_train["CuStepNo ValueY"]==3)&(df_train["Batch"]==2)].drop(["DeviationID ValueY","Batch","Next_Step","timestamp"],axis=1).values

In [25]:
d = dtw_ndim.distance(df_3,df_2)

In [26]:
d

14831.1583363423

In [13]:
ds

array([[0., 0.],
       [0., 0.]])

In [8]:
liste2 = [df_2,df_3]

In [26]:
cdist_dtw(df_4,liste2)

ValueError: All input time series must have the same feature size.

In [38]:
y_predicted

[9]